<a href="https://colab.research.google.com/github/sugi-san/3d-movie/blob/master/3d_ken_burns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **1.セットアップ**

# ライブラリーのインストール
!pip install chainer
!pip install moviepy
!pip install gevent

# githubからコードを取得
!git clone https://github.com/sugi-san/3d-movie.git
%cd /content/3d-movie

# ディレクトリの作成
!mkdir output

# chainerセッティング
import chainer
import os
chainer.print_runtime_info()
os.environ['CUDA_HOME'] = "/usr/local/cuda"

# 学習済みモデルのダウンロード
!wget -O ./models/disparity-estimation.pytorch http://content.sniklaus.com/kenburns/network-disparity.pytorch
!wget -O ./models/disparity-refinement.pytorch http://content.sniklaus.com/kenburns/network-refinement.pytorch
!wget -O ./models/pointcloud-inpainting.pytorch http://content.sniklaus.com/kenburns/network-inpainting.pytorch

# パラメータのダウンロード
import torch
import torchvision
import cupy
exec(open('./common.py', 'r').read())
exec(open('./models/disparity-estimation.py', 'r').read())
exec(open('./models/disparity-adjustment.py', 'r').read())
exec(open('./models/disparity-refinement.py', 'r').read())
exec(open('./models/pointcloud-inpainting.py', 'r').read())


# ライブラリのインポート
import torch
import torchvision
import base64
import cupy
import cv2
import flask
import getopt
import gevent
import gevent.pywsgi
import glob
import h5py
import io
import math
import moviepy
import moviepy.editor
import numpy
import os
import random
import re
import scipy
import scipy.io
import shutil
import sys
import tempfile
import time
import urllib
import zipfile

assert(int(str('').join(torch.__version__.split('.')[0:2])) >= 12) # requires at least pytorch version 1.2.0
torch.set_grad_enabled(False) # make sure to not compute gradients for computational performance
torch.backends.cudnn.enabled = True # make sure to use cudnn for computational performance
objCommon = {}


# 関数定義
from function import *

# ダウンロードフォルダ作成
reset_folder('download')


In [ ]:
#@title **2.写真の表示**
display_pic('input')

In [ ]:
#@title **3.動画の作成**
#@markdown shift : （基準：100）視点と写真の中心点との差\
#@markdown distance : （基準：1.25）視点の前方への移動距離

input = '01.jpg' #@param {type:"string"}
arguments_strIn = './input/'+input
arguments_strOut = './output/movie.mp4'
 
shift =  100#@param {type:"slider", min:0, max:200, step:10}
distance = 1.25  #@param {type:"slider", min:1.25, max:2, step:0.05}
end = 4 
step = int(160*distance - 20)

##########################################################
npyImage = cv2.imread(filename=arguments_strIn, flags=cv2.IMREAD_COLOR)

intWidth = npyImage.shape[1]
intHeight = npyImage.shape[0]

fltRatio = float(intWidth) / float(intHeight)

intWidth = min(int(1024 * fltRatio), 1024)
intHeight = min(int(1024 / fltRatio), 1024)

# ----------
if not intWidth % 2 == 0:
  intWidth +=1

if not intHeight % 2 ==0:
  intHeight +=1
# ----------

npyImage = cv2.resize(src=npyImage, dsize=(intWidth, intHeight), fx=0.0, fy=0.0, interpolation=cv2.INTER_AREA)

process_load(npyImage, {})

objFrom = {
		'fltCenterU': intWidth / 2.0,
		'fltCenterV': intHeight / 2.0,
		'intCropWidth': int(math.floor(0.97 * intWidth)),
		'intCropHeight': int(math.floor(0.97 * intHeight))
}

objTo = process_autozoom({
		'fltShift': shift,  
		'fltZoom': distance, 
		'objFrom': objFrom
})

npyResult = process_kenburns({
		'fltSteps': numpy.linspace(0.0, end, step).tolist(),
		'objFrom': objFrom,
		'objTo': objTo,
		'boolInpaint': True
})

moviepy.editor.ImageSequenceClip(sequence=[ npyFrame[:, :, ::-1] for npyFrame in npyResult + list(reversed(npyResult))[1:] ], fps=25).write_videofile(arguments_strOut)

# ログ・クリア
from IPython.display import clear_output
clear_output()

# 動画再生
display_mp4('./output/movie.mp4')

In [ ]:
#@title **4.動画のダウンロード**
import os
import shutil
from google.colab import files

download_path = 'download/'+os.path.splitext(input)[0]+'.mp4'
shutil.copy('output/movie.mp4', download_path)
files.download(download_path)


In [ ]:
#@title **5.写真のアップロード**

# ルートへ画像をアップロード
from google.colab import files
uploaded = files.upload()           
uploaded = list(uploaded.keys())    

# ルートから指定フォルダーへ移動
for file in uploaded:
    shutil.move(file, 'input')